## 大模型答案生成（RAG-Generation）

大模型根据检索内容生成的原理：把检索到的【知识库内容】、【用户问题】、【回答要求】一起给大模型，让大模型按照你的要求来回答

<img src="/data/xieyu/Teaching/RAG/imgs/rag-in-action.jpeg" alt="向量检索流程图" width="60%" style="display: block; margin: auto;">

## 基础实现

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# 初始化RAG工作流
template_rag = """
你是一个知识库回答专家，请你根据用户的问题，从知识库中找到最相关的信息，整理抽取出对应的详细信息，要求回复简洁，每次回答后提醒我要检查我的回答是否正确，如果回答错误，请重新回答

检索到的内容：{context}
用户问题：{query}
    """
## 这里使用了langchain的lcel语法，如果没有学习过的可以取查看langchain的官方文档
gpt = ChatOpenAI(model="gpt-4o-eus2", api_key="your_key", base_url="your_base_url")
rag_template= PromptTemplate.from_template(template_rag)
rag = {"context": RunnablePassthrough(), "query": RunnablePassthrough()} | rag_template | gpt

# 配置embeding模型
embeding_model_path = "/data/lilk/RAG/models/embeding/bge-m3"
# 配置运行的参数
embeding_model_kwargs = {"device": "cuda"}
embeding_encode_kwargs = {"normalize_embeddings": True}
embeding_model = HuggingFaceEmbeddings(model_name = embeding_model_path,
                                       model_kwargs=embeding_model_kwargs,
                                       encode_kwargs=embeding_encode_kwargs)
vector_store = FAISS.load_local(index_name="rag_teach",folder_path="/data/xieyu/Teaching/RAG/data_retriever/rag_teach",embeddings=embeding_model,allow_dangerous_deserialization=True )

# 执行检索
rag_content = vector_store.similarity_search("RAG是什么？")
result = rag.invoke({"context": rag_content, "query": "RAG是什么？"}).content
print(result)

RAG（Retrieval-Augmented Generation）是一种能让大量知识“活”起来，人与知识无障碍对话的技术。它的背景是，虽然AI大模型已经能够和用户流畅交流，解读知识，总结文章，但由于训练它们的知识主要是互联网过往的公开内容，因此当我们有非公开的知识需要AI帮忙解读、总结时，这些大模型会遇到知识盲区。RAG技术的作用是让这些大模型能够了解到我们的非公开知识，从而更加精准地回答我们的问题。

请检查您的回答是否正确，如果回答错误，请重新回答。


## 代码优化

In [10]:
# 使用RunnableParallel来优化检索和回答的流程
# 将检索和回答合并为一个完整的RAG工作流

from langchain_core.runnables import RunnableParallel, RunnablePassthrough
# 设置并行运行的检索和问题传递
setup_and_retrieval = RunnableParallel(
    {"context": vector_store.as_retriever(), 
     "query": RunnablePassthrough()}
)

# 构建完整的RAG链
rag_chain = setup_and_retrieval | rag_template | gpt 

# 执行RAG查询
result = rag_chain.invoke("RAG是什么？")
print(result)

content='RAG是一种能让大量知识“活”起来，人与知识无障碍对话的技术。随着AI大模型的发展，它们已经能够与用户流畅交流，解读知识并总结文章。但是，由于这些模型的训练数据主要来源于互联网的公开内容，当涉及到非公开知识时，AI模型会遇到知识盲区。因此，RAG技术的目标是让AI模型能够获取并解读非公开知识，从而更加精准地回答用户的问题。\n\n请检查我的回答是否正确，如果回答错误，请重新回答。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 742, 'total_tokens': 859}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'stop', 'logprobs': None} id='run-d745f390-41a0-40cb-994d-24e2dad61d83-0' usage_metadata={'input_tokens': 742, 'output_tokens': 117, 'total_tokens': 859}


## 知识库记忆功能

有时候我们提问知识库，但是希望多轮回答，则需要让这个知识库带有记忆功能

In [24]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory

def get_rephrase_prompt():
    system_prompt = """\
    鉴于以下对话和后续问题，将后续问题重新表述为独立问题。
    """
    rephrase_prompt = ChatPromptTemplate([
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ])
    return rephrase_prompt

def get_rag_prompt():
    system_prompt = """\
    你是一个知识库回答专家，请你根据用户的问题，从知识库中找到最相关的信息，整理抽取出对应的详细信息，要求回复简洁，每次回答后提醒我要检查我的回答是否正确，如果回答错误，请重新回答
    知识库内容如下：
    {context}
    """
    rag_template = ChatPromptTemplate([
        MessagesPlaceholder("chat_history"),
    ("system", system_prompt),
        ("human", "{input}")
    ])
    return rag_template

rephrase_template = get_rephrase_prompt()
rag_template = get_rag_prompt()

# 定义带历史记录的检索器
history_aware_retriever = create_history_aware_retriever(gpt, vector_store.as_retriever(), rephrase_template)
rag_chain = create_stuff_documents_chain(gpt, rag_template)
# 创建检索链
rag_history_chain = create_retrieval_chain(history_aware_retriever, rag_chain)
# 添加消息历史记录功能 
store = {}
def get_session_history(session_id:str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 创建带历史记录的RAG链
conversational_rag_chain = RunnableWithMessageHistory(
    rag_history_chain,
    get_session_history, 
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)
# 测试带记忆的RAG
result = conversational_rag_chain.invoke(
    {"input": "RAG是什么?"}, 
    config={"configurable": {"session_id": "test123"}}
)
print(result["answer"])
print("-"*100)
# 测试第二个问题,会带上上下文
result = conversational_rag_chain.invoke(
    {"input": "它有什么优点?"}, 
    config={"configurable": {"session_id": "test123"}}
)
print(result["answer"])

RAG是一种能让大量知识 “活” 起来，人与知识无障碍对话的技术。AI大模型已经能够和用户流畅交流，解读知识，总结文章，但由于训练他们的知识是互联网过往的公开内容，当我们有非公开的知识想让AI帮忙解读，总结时就涉及到他们的知识盲区，所以RAG技术就是让他们能够了解到我们的非公开知识，更加精准地回答我们的问题。

请检查我的回答是否正确，如果回答错误，请重新回答。
----------------------------------------------------------------------------------------------------
RAG技术有以下优点：

1. **知识活化**：RAG可以让大量知识“活”起来，使人与知识无障碍对话，提供更加流畅的交流体验。
2. **精准回答**：通过结合非公开知识，RAG技术能够更精准地回答用户的问题，解决AI大模型在非公开知识上的盲区。
3. **多场景应用**：RAG技术可以应用在企业知识库、客服机器人、产品自动推荐等多个场景中。
4. **高效检索**：RAG可以通过多种方式（如大模型改写、向量检索等）高效地检索所需信息，提升查询效率。
5. **灵活集成**：RAG技术的多种应用方案（如langchain、dbgpt、ragflow等）使其能够灵活集成到不同的系统中，满足多样化的需求。

请检查我的回答是否正确，如果回答错误，请重新回答。


## 总结


1. 大模型答案生成，需要把【知识库内容】、【用户问题】、【回答要求】一起给大模型，让大模型按照你的要求来回答
2. 大模型答案生成，可以使用langchain的lcel语法，也可以使用langchain的RunnablePassthrough来优化检索和回答的流程
3. 大模型答案生成，可以使用RunnableParallel来优化检索和回答的流程
4. 大模型答案生成，可以使用RunnableWithMessageHistory来实现知识库记忆功能

很多小伙伴看到这里会开始疑惑说那如果知识库需要联网呢？
不知道大家有没有思考过一个问题？如果网络上也有你想要提问的问题的答案呢？但是跟你的知识库存在多个版本？这种情况应该以哪个为主呢？这就会产生很多回答准确度的问题，所以你如果你对这个感兴趣就可以再看看下一张，rag_agent，会有更多方案进行介绍

## dify报告优化

In [1]:
data_trend = """发布时间	数量	点赞数	评论数	收藏数	分享数
11/1/2024	134.0	137656.0	33669.0	8976.0	100798.0
11/11/2024	44.0	1731.0	575.0	182.0	1423.0
11/13/2024	136.0	2069.0	814.0	224.0	5165.0
11/15/2024	38.0	381.0	141.0	229.0	1092.0
11/17/2024	29.0	554.0	100.0	269.0	2339.0
11/19/2024	68.0	790.0	328.0	308.0	1751.0
11/21/2024	113.0	1766.0	550.0	506.0	9312.0
11/23/2024	79.0	545.0	289.0	400.0	1464.0
11/25/2024	90.0	706.0	177.0	240.0	2734.0
11/27/2024	59.0	364.0	188.0	159.0	1108.0
11/29/2024	134.0	1839.0	105.0	327.0	844.0
12/3/2024	58.0	9653.0	1619.0	877.0	201950.0
12/5/2024	54.0	42505.0	21256.0	4662.0	58481.0
12/7/2024	39.0	4350.0	926.0	932.0	1142.0
12/9/2024	40.0	1836.0	737.0	953.0	11917.0
12/11/2024	102.0	998.0	494.0	439.0	3335.0"""

data_summary = """
        平台: 公众号, 内容: 阿斯利康与第一三共合作的药物 datopotamab deruxtecan 获得了首个突破性疗法认定，标志着该药物在治疗特定癌症方面取得了重要进展。, 点赞数: 0, 评论数: 0.0, 分享数: 0, 浏览量: 2.0
        平台: 公众号, 内容: 阿斯利康宣布“笃行志远保留计划”，发放4.9亿奖金，以安抚员工并留住人才。此前公司因涉嫌骗保等事件影响，进行了多条产品线的裁员，赔偿方案为N+3。尽管面临挑战，阿斯利康在中国区的业绩依然出色。, 点赞数: 3, 评论数: 0.0, 分享数: 17, 浏览量: 318.0
        平台: 公众号, 内容: 阿斯利康的AZD2936和DS-8201a获得中国国家药监局临床试验许可，为HER2表达胆道癌患者提供新的治疗选择，增强免疫反应并提高疗效。, 点赞数: 0, 评论数: 0.0, 分享数: 8, 浏览量: 80.0
        平台: 公众号, 内容: 阿斯利康计划明年为员工加薪2-3K，主要体现在固定底薪上，涨幅约为20%。, 点赞数: 0, 评论数: 0.0, 分享数: 0, 浏览量: 58.0
        平台: 公众号, 内容: 阿斯利康的ADC药物DS-8201a和双特异性抗体AZD2936在中国获得临床试验批准，用于治疗HER2阳性胆道癌。这两款药物的联合疗法显示出创新性和潜在疗效。, 点赞数: 0, 评论数: 0.0, 分享数: 9, 浏览量: 176.0
        平台: 公众号, 内容: 处理失败: Expecting value: line 1 column 1 (char 0), 点赞数: 1, 评论数: 0.0, 分享数: 21, 浏览量: 2368.0
        平台: 公众号, 内容: 阿斯利康宣布将给予员工高额激励，包括经理以上20%年薪的激励和经理以下10%年薪的激励，以及Q4的“齐心并进奖”。此举旨在稳定员工情绪，提升销售业绩。, 点赞数: 8, 评论数: 1.0, 分享数: 357, 浏览量: 15000.0
        平台: 公众号, 内容: 阿斯利康再次发放奖金，经理以下员工获得年薪10%现金，经理以上获得20%股权激励。此前公司已发放大额奖金，奖励金额按业绩表现确定，旨在鼓励员工齐心并进。, 点赞数: 30, 评论数: 15.0, 分享数: 1060, 浏览量: 21000.0
        平台: 视频号, 内容: 文章讨论了阿斯利康近期裁员对员工就业前景的负面影响，质疑公司在当前经济环境下裁员的合理性。, 点赞数: 48, 评论数: 16.0, 分享数: 246, 浏览量: nan
        平台: 视频号, 内容: 跨国医药巨头阿斯利康因涉嫌骗保行为，导致公司获利而员工面临刑事处罚。, 点赞数: 14, 评论数: 10.0, 分享数: 10, 浏览量: nan
        平台: 视频号, 内容: 文章质疑阿斯利康涉及诈骗案，暗示政府欠款不还，导致企业被纪检委调查。, 点赞数: 14, 评论数: 15.0, 分享数: 16, 浏览量: nan
        平台: 视频号, 内容: 阿斯利康中国总裁因医保诈骗案被捕，股价下跌，多名员工被立案调查，涉及伪造体检报告以实现医保骗保和高利润，持续三年。, 点赞数: 4, 评论数: 1.0, 分享数: 7, 浏览量: nan
        平台: 视频号, 内容: 阿斯利康因泰瑞莎药物的不当使用和伪造病历行为被曝光，涉及骗保问题。, 点赞数: 0, 评论数: 1.0, 分享数: 2, 浏览量: nan
        平台: 视频号, 内容: 睿昂基因实控人熊慧、熊钧因涉嫌诈骗罪被批捕，可能与阿斯利康骗保案有关，公司与阿斯利康在基因检测方面有深度合作。, 点赞数: 14, 评论数: 2.0, 分享数: 5, 浏览量: nan
        平台: 小红书, 内容: 阿斯利康中国肿瘤业务年会宣布了一项名为“笃行至远保留计划”的专项投入，旨在通过现金和股票奖励留住和激励员工，涉及金额高达10亿元。, 点赞数: 16, 评论数: 6.0, 分享数: 44, 浏览量: nan
        平台: 小红书, 内容: 作者在社交媒体上询问关于选择阿斯利康或晖致的医药代表职位的建议。, 点赞数: 0, 评论数: 0.0, 分享数: 0, 浏览量: nan
        平台: 小红书, 内容: 文章介绍了成都多家外企的工作环境和福利待遇，特别是阿斯利康，强调其优厚的福利和良好的工作环境。, 点赞数: 0, 评论数: 5.0, 分享数: 0, 浏览量: nan
        平台: 小红书, 内容: 阿斯利康宣布将向员工发放奖金，经理以下员工将获得年薪10%的现金，经理以上员工将获得年薪20%的股权激励。, 点赞数: 1, 评论数: 1.0, 分享数: 0, 浏览量: nan
        平台: 小红书, 内容: 阿斯利康中国宣布“笃行志远保留计划”，提供4.9亿现金+股票奖励，旨在安抚员工并保留人才。, 点赞数: 67, 评论数: 26.0, 分享数: 516, 浏览量: nan
        平台: 小红书, 内容: 阿斯利康总部决定发放10亿奖金给员工，引发积极反响。, 点赞数: 51, 评论数: 11.0, 分享数: 311, 浏览量: nan
        平台: 小红书, 内容: 阿斯利康总部批准10亿人民币发放给中国全体员工，体现了公司对员工的重视和奖励。, 点赞数: 10, 评论数: 3.0, 分享数: 87, 浏览量: nan
        平台: 小红书, 内容: 阿斯利康改进了布地格福吸入气雾剂的刻度，提升了用户体验。, 点赞数: 1, 评论数: 0.0, 分享数: 0, 浏览量: nan
        平台: 小红书, 内容: 阿斯利康将在2024年10月7-10号于长沙国际会议中心参加展会，涉及展位设计和搭建。, 点赞数: 7, 评论数: 8.0, 分享数: 0, 浏览量: nan
        平台: 小红书, 内容: 阿斯利康在医药行业逆向涨薪，固定底薪提升2-3K，涨幅约20%，显示其对国内市场的重视。, 点赞数: 10, 评论数: 5.0, 分享数: 9, 浏览量: nan
        平台: 小红书, 内容: 阿斯利康宣布大幅涨薪，涨幅达2-3K，旨在应对医药代表不承担销售指标的新政策，同时提升内部团结和外部吸引力，挽回公司形象。, 点赞数: 10, 评论数: 4.0, 分享数: 40, 浏览量: nan
        平台: 小红书, 内容: 阿斯利康提供留学生投递机会，包括情景题和能力测试在内的OT流程详细，适合对医药和科技感兴趣的学生。, 点赞数: 1, 评论数: 0.0, 分享数: 0, 浏览量: nan
        平台: 小红书, 内容: 阿斯利康参与北京链博会，展示其在多个行业的品牌传播和会展服务，强调其专业资质和全球服务网络。, 点赞数: 3, 评论数: 6.0, 分享数: 0, 浏览量: nan
        平台: 小红书, 内容: 阿斯利康计划为员工加薪2000-3000元，并且近期还发放了股票，显示出公司对员工福利的重视。, 点赞数: 25, 评论数: 8.0, 分享数: 130, 浏览量: nan
        平台: 小红书, 内容: 文章提供了阿斯利康2025届财务管培生终面的真题解析，旨在帮助求职者更好地准备面试。, 点赞数: 1, 评论数: 1.0, 分享数: 0, 浏览量: nan
        平台: 小红书, 内容: 处理失败: Expecting value: line 1 column 1 (char 0), 点赞数: 218, 评论数: 132.0, 分享数: 23, 浏览量: nan
        平台: 小红书, 内容: 艾默生等多家外企在年末进行招聘，提供多种职位和优厚福利，包括13薪、五险一金、弹性工作制等。, 点赞数: 23, 评论数: 33.0, 分享数: 6, 浏览量: nan
        平台: 小红书, 内容: 文章介绍了阿斯利康作为一家综合型生物制药企业，并鼓励有意向进入外企的读者关注作者以获取更多外企信息。, 点赞数: 4, 评论数: 7.0, 分享数: 1, 浏览量: nan
        平台: 小红书, 内容: 文章讨论了EPA乙酯在心血管疾病治疗中的重要性，特别是阿斯利康的96.5% EPA乙酯制剂，能够显著降低心血管不良事件风险，填补了他汀类药物的残余风险。, 点赞数: 4, 评论数: 6.0, 分享数: 1, 浏览量: nan
        平台: 小红书, 内容: 文章讨论了阿斯利康医药代表实习生的简历筛选问题，涉及药企、简历、实习、寒假实习和经管等话题。, 点赞数: 4, 评论数: 0.0, 分享数: 0, 浏览量: nan
        平台: 小红书, 内容: 文章介绍了医药行业外企的薪资高、发展前景好、女性友好等特点，适合医药学专业的学生实习和工作，列举了11家知名医药公司。, 点赞数: 10, 评论数: 5.0, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 默沙东与阿斯利康合作的Keytruda与Lynparza组合在III期卵巢癌试验中取得成功，尽管整体临床试验中鲜有重大成功。, 点赞数: 2, 评论数: nan, 分享数: 1, 浏览量: nan
        平台: 微博, 内容: 阿斯利康公布的III期AMPLIFY研究显示，阿可替尼与维奈克拉联合治疗CLL的总缓解率高达93%，显著优于标准治疗。, 点赞数: 0, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 美FDA授予阿斯利康和第一三共的TROP2抗体偶联药Datopotamab deruxtecan一线治疗局部浸润、EGFR突变阳性的非小细胞肺癌突破治疗地位，基于II期临床研究TROPION-Lung05的积极结果。, 点赞数: 34, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 石药集团执行总裁刘勇军在加盟仅3个月后宣布离职，公司尚未对此消息进行官方回应。, 点赞数: 0, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 石药集团高管刘勇军博士离职，此前曾担任信达生物总裁及阿斯利康等多家公司要职。, 点赞数: 0, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 2024药品‘国谈’中，CAR-T药物第四次被拒，反映出中国CGTs创新药企业在医保报销方面面临的挑战。, 点赞数: 1, 评论数: nan, 分享数: 1, 浏览量: nan
        平台: 微博, 内容: 阿斯利康公布的III期研究显示，其口服BTK抑制剂Calquence显著降低了初治慢性淋巴细胞白血病患者的疾病进展风险，新数据在ASH年会上发布。, 点赞数: 0, 评论数: nan, 分享数: 1, 浏览量: nan
        平台: 微博, 内容: 诺华公司公布新数据，显示其口服补体抑制剂Fabhalta在PNH患者中表现出疗效，提高了治疗的灵活性。, 点赞数: 2, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 睿昂基因实控人姐弟因涉嫌诈骗罪被捕，可能与阿斯利康骗保案有关。, 点赞数: 0, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 阿斯利康员工因骗保案被判刑13年，公司面临法律和声誉风险。, 点赞数: 0, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 阿斯利康中国管理层进行重大调整，文章通过王磊事件探讨了工作狂的负面影响及适时舍弃的重要性。, 点赞数: 0, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 文章提到了Lp(a)相关的多家制药公司，包括礼来、石药/阿斯利康、恒瑞、信立泰和京新，但未提供具体信息或观点。, 点赞数: 0, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 文章提到了Lp(a)相关的多家药企，包括礼来、石药/阿斯利康、恒瑞、信立泰和京新，但未提供具体信息或观点。, 点赞数: 3, 评论数: nan, 分享数: 2, 浏览量: nan
        平台: 微博, 内容: 文章介绍了欧洲股市“八巨头”及其在欧洲斯托克600指数中的重要地位，尽管欧洲经济增长乏力，这些公司仍占据市场重要份额。, 点赞数: 0, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 文章谴责阿斯利康在骗保案中的行为，指出阿斯利康工作人员是事件的始作俑者，同时提到医院工作人员和病人在事件中的角色。, 点赞数: 0, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 文章讨论了H5N1变异新闻对医疗投资的影响，提醒投资者注意选择合适的医疗基金，避免投资表现不佳的基金。, 点赞数: 37, 评论数: nan, 分享数: 3, 浏览量: nan
        平台: 微博, 内容: 美国FDA批准阿斯利康的PD-L1抑制剂度伐利尤单抗用于治疗局限期小细胞肺癌，这是四十年来的重大突破。, 点赞数: 1, 评论数: nan, 分享数: 2, 浏览量: nan
        平台: 微博, 内容: 阿斯利康的PD-L1抑制剂Imfinzi获得FDA批准，用于治疗局限期小细胞肺癌，标志着40年来的重要突破。, 点赞数: 2, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 文章介绍了泰国MMS/CDS社群分享的使用MMS解决疫苗副作用的案例，提到MMS比CDS更有效。, 点赞数: 8, 评论数: nan, 分享数: 2, 浏览量: nan
        平台: 微博, 内容: 诺华公布了其口服补体抑制剂Fabhalta在PNH患者中的新数据，显示该药物在治疗灵活性方面具有潜在改善。, 点赞数: 3, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 文章介绍了阿斯利康公司研发的奥美拉唑肠溶片，这是一种原研药。, 点赞数: 0, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 阿斯利康和第一三共撤回了针对datopotamab deruxtecan在非鳞状非小细胞肺癌中的申请后，公布了支持其在另一群体中重新申请批准的结果。, 点赞数: 2, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 睿昂基因的四位高管因涉嫌诈骗被刑事拘留，涉及阿斯利康骗保案。, 点赞数: 18, 评论数: nan, 分享数: 4, 浏览量: nan
        平台: 微博, 内容: 阿斯利康招聘上海市场部实习生，提供不同学历的薪资标准和餐补，岗位职责包括内容处理、数据分析和沟通协作。, 点赞数: 1, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 阿斯利康在上海招聘市场部实习生，提供具有竞争力的实习薪资和餐补，岗位职责包括内容处理、数据分析和跨部门沟通。, 点赞数: 0, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 阿斯利康的PD-L1抑制剂Imfinzi获得FDA批准，用于治疗未进展的局限期小细胞肺癌成人患者，此前患者已接受同步铂类化疗和放疗。, 点赞数: 2, 评论数: nan, 分享数: 1, 浏览量: nan
        平台: 微博, 内容: 睿昂基因实控人姐弟因涉嫌诈骗罪被正式批捕，涉及阿斯利康“骗保”案。, 点赞数: 0, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 阿斯利康中国进行了管理层的重大调整。, 点赞数: 0, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 阿斯利康宣布了多项人事任命，涉及中国区高层。, 点赞数: 0, 评论数: nan, 分享数: 0, 浏览量: nan
        平台: 微博, 内容: 阿斯利康的PD-L1抑制剂Imfinzi获得FDA批准，用于治疗局限期小细胞肺癌，标志着40年来的重要突破。, 点赞数: 0, 评论数: nan, 分享数: 2, 浏览量: nan
        平台: 抖音, 内容: 文章通过内部图片质疑阿斯利康的行为，表达了对阿斯利康财力的感叹，但未明确表达正面或负面态度。, 点赞数: 22, 评论数: 4.0, 分享数: 39, 浏览量: nan
        平台: 抖音, 内容: 处理失败: Expecting value: line 1 column 1 (char 0), 点赞数: 54, 评论数: 5.0, 分享数: 201, 浏览量: nan
        平台: 抖音, 内容: 文章仅提及大型制药企业，未提供具体信息或观点。, 点赞数: 9, 评论数: 2.0, 分享数: 2, 浏览量: nan
        平台: 抖音, 内容: 文章内容为空，无具体信息可供分析。, 点赞数: 0, 评论数: 0.0, 分享数: 0, 浏览量: nan
        平台: 抖音, 内容: 文章描述了作者因食用小山楂后出现胃疼、恶心等症状，尝试通过喝可乐和吃药片来缓解不适。, 点赞数: 2, 评论数: 0.0, 分享数: 0, 浏览量: nan
        平台: 抖音, 内容: 广州外企12月秋招补录，部分岗位要求宽松，为应届生提供机会。, 点赞数: 1, 评论数: 1.0, 分享数: 1, 浏览量: nan
        平台: 抖音, 内容: 文章提及新会陈皮作为优质农产品原产地，但未提供具体信息或观点。, 点赞数: 26, 评论数: 1.0, 分享数: 0, 浏览量: nan
        平台: 抖音, 内容: 文章表达了作者在经济和家庭方面的困境，以及对未来生活的担忧。, 点赞数: 17, 评论数: 0.0, 分享数: 0, 浏览量: nan
        平台: 门户网站, 内容: 阿斯利康2024年第三季度财报显示，公司收入和每股收益均实现显著增长，中国市场表现尤为突出，成为其全球第二大市场。尽管前负责人王磊被调查，公司仍承诺在中国长期发展并继续投资。, 点赞数: 3, 评论数: 10.0, 分享数: 9, 浏览量: 8.0
        平台: 门户网站, 内容: 阿斯利康CEO苏博科在投资者会议上表示，公司将继续在中国投资并支持业务发展，尽管面临高管被调查的风波。中国业务第三季度收入同比增长15%，占公司整体收入的12.89%。, 点赞数: 3, 评论数: 1.0, 分享数: 3, 浏览量: 9.0
        平台: 门户网站, 内容: 阿斯利康与石药集团达成临床前创新降脂药物授权许可，旨在加强心血管疾病领域的药物管线，解决血脂异常和相关心血管疾病的风险因素。, 点赞数: 8, 评论数: 2.0, 分享数: 4, 浏览量: 4.0
        平台: 门户网站, 内容: 阿斯利康宣布Iskra Reic担任全球执行副总裁、国际业务负责人，同时林骁出任阿斯利康中国总经理。王磊因在中国接受调查而暂离岗位，阿斯利康表示调查仅针对个人，不涉及公司层面。, 点赞数: 10, 评论数: 6.0, 分享数: 7, 浏览量: 4.0
        平台: 门户网站, 内容: 阿斯利康宣布新的领导层变动，Iskra Reic接任国际业务负责人，林骁和关冬梅分别担任中国总经理和中国肿瘤业务总经理。尽管在中国市场面临调查和负面消息，阿斯利康仍保持对中国市场的重视，并对2025年市场拓展充满信心。, 点赞数: 0, 评论数: 10.0, 分享数: 0, 浏览量: 0.0
        平台: 门户网站, 内容: 阿斯利康宣布Iskra Reic接替王磊担任国际业务负责人，中国团队进行调整，王磊因接受调查暂离岗位。, 点赞数: 1, 评论数: 7.0, 分享数: 3, 浏览量: 9.0
        平台: 门户网站, 内容: 阿斯利康任命Iskra Reic为全球执行副总裁、国际业务负责人，负责推动国际业务的可持续增长。Iskra在多个领域拥有丰富经验，曾成功领导疫苗和免疫疗法业务部。, 点赞数: 1, 评论数: 6.0, 分享数: 6, 浏览量: 0.0
        平台: 门户网站, 内容: 阿斯利康宣布人事调整，林骁出任中国总经理。, 点赞数: 3, 评论数: 8.0, 分享数: 1, 浏览量: 7.0
        平台: 门户网站, 内容: 阿斯利康宣布Iskra Reic接任国际业务负责人，林骁出任中国总经理。王磊因在中国接受调查而暂离岗位，公司表示调查仅针对个人，不涉及公司层面。阿斯利康强调在中国的业务和合规政策，中国区业务增长良好。, 点赞数: 3, 评论数: 4.0, 分享数: 7, 浏览量: 5.0
        平台: 门户网站, 内容: 阿斯利康宣布林骁将接替赖明隆担任中国总经理，林骁目前担任阿斯利康中国生物制药业务总经理及中国香港、澳门地区负责人。, 点赞数: 2, 评论数: 2.0, 分享数: 3, 浏览量: 2.0
        平台: 门户网站, 内容: 阿斯利康进行中国人事调整，林骁出任中国总经理，关冬梅出任中国肿瘤业务总经理，多位高管直接向国际业务负责人汇报。, 点赞数: 9, 评论数: 4.0, 分享数: 6, 浏览量: 5.0
        平台: 门户网站, 内容: 阿斯利康中国管理层因王磊接受调查而进行大调整，Iskra Reic接任国际业务负责人，林骁成为中国总经理，赖明隆转任全球肿瘤业务负责人。, 点赞数: 4, 评论数: 8.0, 分享数: 3, 浏览量: 7.0
        平台: 门户网站, 内容: 阿斯利康宣布多项人事调整，Iskra Reic接替王磊担任全球执行副总裁、国际业务负责人，同时启动中国区管理团队的人事变动。, 点赞数: 8, 评论数: 1.0, 分享数: 3, 浏览量: 2.0
        平台: 门户网站, 内容: 阿斯利康宣布Iskra Reic接替因在中国接受调查而暂离岗位的王磊，担任全球执行副总裁、国际业务负责人，负责多个地区的整体战略。阿斯利康表示将全力配合调查。, 点赞数: 6, 评论数: 0.0, 分享数: 10, 浏览量: 1.0
        平台: 门户网站, 内容: 阿斯利康中国进行人事调整，林骁出任中国总经理，关冬梅出任中国肿瘤业务总经理，赖明隆担任全球肿瘤业务Dato-DXd及肺癌事业部负责人。, 点赞数: 2, 评论数: 8.0, 分享数: 7, 浏览量: 6.0
        平台: 门户网站, 内容: 阿斯利康宣布Iskra Reic接任全球执行副总裁、国际业务负责人，同时宣布中国团队高管任命。王磊因调查事件离职，Iskra将负责推动国际业务增长，中国区收入持续增长。, 点赞数: 9, 评论数: 7.0, 分享数: 8, 浏览量: 1.0
        平台: 门户网站, 内容: 阿斯利康宣布多项高层人事调整，包括Iskra Reic接替国际业务负责人，林骁接替中国总经理，关冬梅接任中国肿瘤业务总经理。Iskra Reic强调中国市场的重要性，并表示将全力配合政府调查。, 点赞数: 1, 评论数: 2.0, 分享数: 7, 浏览量: 9.0
        平台: 门户网站, 内容: 睿昂基因实控人姐弟熊慧、熊钧因涉嫌诈骗被批捕，公司股价下跌，多位高管辞职。事件与阿斯利康骗保案有关，公司未来前景堪忧。, 点赞数: 0, 评论数: 10.0, 分享数: 2, 浏览量: 5.0
        平台: 门户网站, 内容: 阿斯利康因涉嫌篡改基因检测报告以骗取医保基金，引发多起骗保案，涉及金额约三千万元，多名员工被判刑，公司股价大幅下跌。, 点赞数: 10, 评论数: 8.0, 分享数: 10, 浏览量: 7.0
        平台: 门户网站, 内容: 处理失败: Expecting value: line 1 column 1 (char 0), 点赞数: 10, 评论数: 0.0, 分享数: 4, 浏览量: 9.0
        平台: 门户网站, 内容: 阿斯利康宣布多项高层人事调整，包括Iskra Reic接任国际业务负责人，林骁接任中国总经理，关冬梅接任中国肿瘤业务总经理。这些调整反映了公司对全球市场的重视和对中国市场的持续关注。, 点赞数: 7, 评论数: 8.0, 分享数: 4, 浏览量: 2.0
        平台: 门户网站, 内容: 阿斯利康中国管理层因王磊被调查而进行大调整，Iskra Reic接任国际业务负责人，林骁成为中国总经理，赖明隆转任全球肿瘤业务负责人。, 点赞数: 6, 评论数: 4.0, 分享数: 5, 浏览量: 9.0
        平台: 门户网站, 内容: 阿斯利康的两款新药AZD2936和DS-8201a在中国获得临床试验批准，旨在治疗HER2阳性胆道癌。DS-8201a为抗体偶联药物，AZD2936为双特异性抗体，二者联合疗法已获中国监管机构批准。, 点赞数: 8, 评论数: 7.0, 分享数: 10, 浏览量: 7.0
        平台: 门户网站, 内容: 阿斯利康在2024年12月7日的学术会议上展示了其在肿瘤免疫治疗领域的最新进展，强调了其创新产品的疗效和安全性，并获得了与会专家的高度评价。, 点赞数: 8, 评论数: 0.0, 分享数: 0, 浏览量: 3.0
        平台: 门户网站, 内容: 阿斯利康任命Iskra Reic为全球执行副总裁、国际业务负责人，接替因在中国接受调查而暂离岗位的王磊。公司强调对中国市场的长期承诺和持续投资。, 点赞数: 2, 评论数: 9.0, 分享数: 10, 浏览量: 8.0
        平台: 门户网站, 内容: 阿斯利康与安进合作的TSLP单抗在中国申报上市，成为国内首个此类药物。, 点赞数: 8, 评论数: 10.0, 分享数: 4, 浏览量: 7.0
        平台: 门户网站, 内容: 赛诺菲在亦庄增资扩产，投资10亿欧元设立全新生产制造基地，彰显了北京亦庄良好的营商环境和对外资企业的吸引力。亦庄通过优质服务和政策支持，增强了赛诺菲在华投资的信心。, 点赞数: 6, 评论数: 5.0, 分享数: 4, 浏览量: 6.0
        平台: 门户网站, 内容: 阿斯利康的ADC药物datopotamab deruxtecan获得FDA突破性疗法认定，用于治疗EGFR突变的非小细胞肺癌，显示出良好的临床效果。, 点赞数: 4, 评论数: 4.0, 分享数: 4, 浏览量: 3.0
        平台: 门户网站, 内容: 阿斯利康的AKT抑制剂Capivasertib在治疗前列腺癌的III期研究中取得积极结果，显示出显著延长放射学无进展生存期。Capivasertib已获FDA和欧盟批准，用于治疗特定类型的乳腺癌。, 点赞数: 7, 评论数: 2.0, 分享数: 3, 浏览量: 10.0
"""

platform_data = """
平台	数量	点赞数	评论数	收藏数	分享数
公众号	8	42	16.0	38	1472
小红书	21	466	267.0	174	1168
微博	30	1116	（NULL）	25	19
抖音	8	131	13.0	50	243
视频号	6	94	45.0	24	286
门户网站	29	149	153.0	128	147
"""

report_date = "2024-12-11"

In [2]:
api_key = "app-r2r2Cq3VSeYd1qvqKtbb0GIG"
query = "帮我分析找一份报告"
params = {
"data_trend":data_trend,
"data_summary":data_summary,
"platform_data":platform_data,
"report_date":report_date
}
files = [
]


In [8]:
import aiohttp
import json
import asyncio
from IPython.display import display, clear_output

async def post_dify_message_stream(api_key, query, params=None, files=None, conversation_id="", user="default_user"):
    """
    异步流式向Dify API发送聊天消息并实时输出响应
    """
    url = "http://localhost:15880/v1/chat-messages"
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "inputs": params or {},
        "query": query,
        "response_mode": "streaming",
        "conversation_id": conversation_id,
        "user": user
    }
    
    if files:
        payload["files"] = files
        
    full_response = ""  # 用于累积完整响应
    
    try:
        async with aiohttp.ClientSession() as session:
            async with session.post(url, headers=headers, json=payload) as response:
                if response.status != 200:
                    error_text = await response.text()
                    raise Exception(f"API请求失败: {response.status} - {error_text}")
                
                # 读取并解析流式响应
                async for line in response.content:
                    if line:
                        text = line.decode('utf-8').strip()
                        if text.startswith('data: '):
                            text = text[6:]  # 移除 "data: " 前缀
                            
                            if text == '[DONE]':
                                break
                                
                            try:
                                data = json.loads(text)
                                if 'answer' in data:
                                    full_response += data['answer']
                                    clear_output(wait=True)
                                    display(full_response)
                            except json.JSONDecodeError:
                                continue
                                
    except Exception as e:
        display(f"错误: {str(e)}")
    
    return full_response

# 使用示例
async def analyze_report(api_key, query, params, files=None):
    response = await post_dify_message_stream(
        api_key=api_key,
        query=query,
        params=params,
        files=files
    )
    return response

# 运行代码
# 你的参数
api_key = "app-r2r2Cq3VSeYd1qvqKtbb0GIG"
query = "帮我分析找一份报告"
params = {
    "data_trend": data_trend,
    "data_summary": data_summary,
    "platform_data": platform_data,
    "report_date": report_date
}
files = []

# 在 Jupyter Notebook 中运行
md_report = await analyze_report(api_key, query, params, files)

from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import re

def markdown_to_word(markdown_text, output_file='report.docx'):
    """
    将 Markdown 格式文本转换为 Word 文档
    
    Args:
        markdown_text (str): Markdown 格式的文本
        output_file (str): 输出的 Word 文件名
    """
    # 创建新的 Word 文档
    doc = Document()
    
    # 定义标题样式
    heading_styles = {
        '####': 'Heading 4',  # 四级标题
        '###': 'Heading 3',   # 三级标题
        '##': 'Heading 2',    # 二级标题
        '#': 'Heading 1'      # 一级标题
    }
    
    # 分割文本为行
    lines = markdown_text.split('\n')
    
    for line in lines:
        # 跳过空行
        if not line.strip():
            doc.add_paragraph()
            continue
            
        # 处理标题
        is_heading = False
        for heading_mark, style_name in heading_styles.items():
            if line.strip().startswith(heading_mark + ' '):
                # 移除标题标记并添加标题
                title_text = line.strip()[len(heading_mark + ' '):]
                heading = doc.add_paragraph(title_text, style=style_name)
                is_heading = True
                break
        
        # 如果不是标题，则作为普通段落处理
        if not is_heading:
            # 处理粗体文本 (**text**)
            bold_pattern = r'\*\*(.*?)\*\*'
            line_parts = re.split(bold_pattern, line)
            
            p = doc.add_paragraph()
            for i, part in enumerate(line_parts):
                if i % 2 == 0:  # 普通文本
                    p.add_run(part)
                else:  # 粗体文本
                    p.add_run(part).bold = True
    
    # 保存文档
    doc.save(output_file)
    return f"文档已保存为 {output_file}"

# 使用示例
def convert_report_to_word(markdown_text):
    """
    转换报告并返回状态
    """
    try:
        result = markdown_to_word(markdown_text, 'media_coverage_report.docx')
        return result
    except Exception as e:
        return f"转换失败: {str(e)}"
status = convert_report_to_word(md_report)
print(status)   

'### Comprehensive Media Coverage Analysis Report\n\n#### 1. Overall Trend Analysis\n\nThe latest media coverage data reveals a significant shift in engagement metrics compared to previous periods. Notably, the period from November 1 to December 11, 2024, shows a marked increase in share counts and a corresponding rise in likes and comments, particularly in the latter half of November and early December. This surge can be attributed to several key factors:\n\n- **Strategic Announcements**: The release of major news, such as the breakthrough therapy designation for datopotamab deruxtecan and the "笃行志远保留计划," has driven substantial engagement. These announcements not only highlight the company\'s advancements in cancer treatment but also its commitment to employee welfare, which resonates well with both industry professionals and the general public.\n- **Industry Events**: Participation in high-profile events, such as the academic conference on December 7, 2024, and the Beijing链博会, has am

文档已保存为 media_coverage_report.docx
